<a href="https://colab.research.google.com/github/datvodinh10/project-DD/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install gdown
!gdown 1dQ7dqVnBfp4STYMVsnkiLfiAIdFwrSkd # Training data
!gdown 1YedVnk4uKFBPInsa6Mzik0bmZK2Vuf4e # Target label data
#!gdown model_weight gdrive id 
!git clone https://github.com/datvodinh10/project-DD.git
%cd project-DD
from src.model.trainer import Trainer
%cd ..
# !unzip -q /content/training_data.zip
# SRC_PATH = "/content/new_train"
# TARGET_PATH = "/content/train_gt.txt"

!unzip -q /kaggle/working/training_data.zip
SRC_PATH = "/kaggle/working/new_train"
TARGET_PATH = "/kaggle/working/train_gt.txt"

In [ ]:
import torch

In [ ]:
from src.model.trainer import Trainer
from src.utils.load_config import load_config

## Train

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [1]:
config = load_config('resnet')
# MODEL_PATH = "/content"
MODEL_PATH = f"/kaggle/working/model_{config['encoder']['type']}_{config['num_epochs']}.pt"

In [ ]:
trainer = Trainer(config      = config,
                  IMAGE_PATH  = SRC_PATH,
                  TARGET_PATH = TARGET_PATH,
                  MODEL_PATH  = MODEL_PATH,
                  device      = device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Total parameters: {count_parameters(trainer.model)}')

In [ ]:
trainer.train()